In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#Import das Bibliotecas
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import ast
import json
import nltk
import numpy as np
import gensim
from pprint import pprint
nltk.download('punkt')
nltk.download('stopwords')
import seaborn as sns
from string import punctuation
from nltk import word_tokenize
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import gensim.corpora as corpora
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from gensim.models import CoherenceModel
from sklearn.decomposition import TruncatedSVD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import spacy
!python -m spacy download pt_core_news_lg

2023-04-19 12:38:33.843216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import gensim.corpora as corpora

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
#Leitura do novo Dataset após tratamento inicial
url = 'https://raw.githubusercontent.com/leonardoemerson/Fake-Dataset/main/dataset/treated_dataset3.csv'
df = pd.read_csv(url, encoding = 'utf8', delimiter = ';', header = 0)

In [ ]:
#Eliminando coluna 'Unnamed: 0'
df = df.drop("Unnamed: 0",axis=1)

In [ ]:
df.shape

(5201, 16)

In [ ]:
#Visualização inicial do Dataset
df.head()

,title,title_norm,message,message_norm,tokens,features,entities,type,source,url_review,message_norm_treatment,datetime,message_norm_treatment_ssw,message_norm_treatment_ssw2,features2,category
0,"Pastor Sérgio Von Helder, que chutou santa, nã...","pastor sergio von helder, que chutou santa, na...",NaN,NaN,"['pastor', 'sergio', 'von', 'helder', 'que', '...","{'len': 12, 1: {'token': 'pastor', 'lemma': 'p...",[Pastor Sérgio Von Helder],fake,boatos.org,https://www.boatos.org/religiao/pastor-sergio-...,NaN,2013-08-07T19:27:23-03:00,[],[],"{'len': 12, 1: {'token': 'pastor', 'lemma': 'p...",Religião
1,"Lulinha, filho de Lula, comprou avião Gulfstre...","lulinha, filho de lula, comprou aviao gulfstre...",NaN,NaN,"['lulinha', 'filho', 'de', 'lula', 'comprou', ...","{'len': 14, 1: {'token': 'lulinha', 'lemma': '...","[Lula, Gulfstream]",fake,boatos.org,https://www.boatos.org/politica/lulinha-aviao-...,NaN,2013-08-02T19:56:26-03:00,[],[],"{'len': 14, 1: {'token': 'lulinha', 'lemma': '...",Política
2,Lulinha não comprou Fazenda Fortaleza por 47 m...,lulinha nao comprou fazenda fortaleza por 47 m...,O Novo Mega Campeão do Brasil de enriqueciment...,o novo mega campeao do brasil de enriqueciment...,"['lulinha', 'nao', 'comprou', 'fazenda', 'fort...","{'len': 8, 1: {'token': 'lulinha', 'lemma': 'l...","[Lulinha, Fazenda]",fake,boatos.org,https://www.boatos.org/politica/lulinha-fazend...,o novo mega campeao do brasil de enriqueciment...,2013-07-29T14:13:21-03:00,"['novo', 'mega', 'campeao', 'brasil', 'enrique...","['novo', 'mega', 'campeao', 'brasil', 'enrique...","{'len': 8, 1: {'token': 'lulinha', 'lemma': 'l...",Política
3,Fanta Uva causa câncer e problema nos rins,fanta uva causa cancer e problema nos rins,ALERTA GERAL ATENÇÃO” FANTA UVA!!!! A propagan...,alerta geral atencao fanta uva!!!!a propaganda...,"['fanta', 'uva', 'causa', 'cancer', 'e', 'prob...","{'len': 8, 1: {'token': 'fanta', 'lemma': 'fan...",[Fanta Uva],fake,boatos.org,https://www.boatos.org/saude/fanta-uva-da-canc...,alerta geral atencao fanta uva!!!!a propaganda...,2013-07-22T04:44:40-03:00,"['alerta', 'geral', 'atencao', 'fanta', 'uva',...","['alerta', 'geral', 'atencao', 'fanta', 'propa...","{'len': 8, 1: {'token': 'fanta', 'lemma': 'fan...",Saúde
4,Simpsons terá episódio sobre manifestações no ...,simpsons tera episodio sobre manifestacoes no ...,NaN,NaN,"['simpsons', 'tera', 'episodio', 'sobre', 'man...","{'len': 7, 1: {'token': 'simpsons', 'lemma': '...","[Simpsons, Brasil]",fake,boatos.org,https://www.boatos.org/entretenimento/simpsons...,NaN,2013-07-08T17:37:40-03:00,[],[],"{'len': 7, 1: {'token': 'simpsons', 'lemma': '...",Entretenimento


In [ ]:
#Ordenando dataset de acordo com a coluna 'datetime'
df = df.sort_values(by='datetime')

In [ ]:
#Resetando os índices do dataset
df = df.reset_index(drop=True)

###COLOCANDO TÍTULO COMO TEXTO PARA NOTÍCIAS SEM TEXTO

In [ ]:
from urllib.request import urlopen
advanced_sw_list = []
file = open("filename","w", encoding='utf-8')
url = urlopen("https://gist.githubusercontent.com/alopes/5358189/raw/2107d809cca6b83ce3d8e04dbd9463283025284f/stopwords.txt")
for line in url:
    file.write(str(line) +'\n')
    advanced_sw_list.append(line)
file.close()

In [ ]:
#Removendo espaços e \n dos elementos da lista avançada de strings(stopwords)
advanced_sw_list1 = list(map(lambda x:x.strip(),advanced_sw_list))

In [ ]:
#Decode para remover o b frente das strings(stopwords) 
for i in range(0, len(advanced_sw_list1)):
    advanced_sw_list1[i] = advanced_sw_list1[i].decode('utf-8')

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 17.0 MB/s eta 0:00:00


In [ ]:
#Removendo acentuação das strings(stopwords)
import unidecode 
for i in range(0, len(advanced_sw_list1)):
    advanced_sw_list1[i] = unidecode.unidecode(advanced_sw_list1[i])

In [ ]:
#Adicionando novas stopwords manualmente
advanced_sw_list1.append('ai')
advanced_sw_list1.append('sobre')
advanced_sw_list1.append('r')
advanced_sw_list1.append('pra')
advanced_sw_list1.append('entao')
advanced_sw_list1.append('ta')
advanced_sw_list1.append('porque')
advanced_sw_list1.append('assim')
advanced_sw_list1.append('onde')
advanced_sw_list1.append('aqui')
advanced_sw_list1.append('agora')
advanced_sw_list1.append('vai')
advanced_sw_list1.append('ainda')
advanced_sw_list1.append('vamos')
advanced_sw_list1.append('diz')
advanced_sw_list1.append('pode')
advanced_sw_list1.append('sendo')
advanced_sw_list1.append('entao')
advanced_sw_list1.append('aqui')
advanced_sw_list1.append('1')

In [ ]:
#Mais stopwords
mais_stopwords = "a, agora, ainda, alguém, algum, alguma, algumas, alguns, ampla, amplas, amplo, amplos, ante, antes, ao, aos, após, aquela, aquelas, aquele, aqueles, aquilo, as, até, através, cada, coisa, coisas, com, como, contra, contudo, da, daquele, daqueles, das, de, dela, delas, dele, deles, depois, dessa, dessas, desse, desses, desta, destas, deste, deste, destes, deve, devem, devendo, dever, deverá, deverão, deveria, deveriam, devia, deviam, disse, disso, disto, dito, diz, dizem, do, dos, e, é, ela, elas, ele, eles, em, enquanto, entre, era, essa, essas, esse, esses, esta, está, estamos, estão, estas, estava, estavam, estávamos, este, estes, estou, eu, fazendo, fazer, feita, feitas, feito, feitos, foi, for, foram, fosse, fossem, grande, grandes, há, isso, isto, já, la, lá, lhe, lhes, lo, mas, me, mesma, mesmas, mesmo, mesmos, meu, meus, minha, minhas, muita, muitas, muito, muitos, na, não, nas, nem, nenhum, nessa, nessas, nesta, nestas, ninguém, no, nos, nós, nossa, nossas, nosso, nossos, num, numa, nunca, o, os, ou, outra, outras, outro, outros, para, pela, pelas, pelo, pelos, pequena, pequenas, pequeno, pequenos, per, perante, pode, pude, podendo, poder, poderia, poderiam, podia, podiam, pois, por, porém, porque, posso, pouca, poucas, pouco, poucos, primeiro, primeiros, própria, próprias, próprio, próprios, quais, qual, quando, quanto, quantos, que, quem, são, se, seja, sejam, sem, sempre, sendo, será, serão, seu, seus, si, sido, só, sob, sobre, sua, suas, talvez, também, tampouco, te, tem, tendo, tenha, ter, teu, teus, ti, tido, tinha, tinham, toda, todas, todavia, todo, todos, tu, tua, tuas, tudo, última, últimas, último, últimos, um, uma, umas, uns, vendo, ver, vez, vindo, vir, vos, vós"
mais_stopwords1 = [ 'a', 'à', 'adeus', 'agora', 'aí', 'ainda', 'além', 'algo', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ali', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'anos', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 'área', 'as', 'às', 'assim', 'até', 'atrás', 'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contra', 'contudo', 'custa', 'da', 'dá', 'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'do', 'dois', 'dos', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'enquanto', 'entre', 'era', 'eram', 'éramos', 'és', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estás', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estiveste', 'estivestes', 'estou', 'etc', 'eu', 'exemplo', 'faço', 'falta', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazes', 'feita', 'feitas', 'feito', 'feitos', 'fez', 'fim', 'final', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foste', 'fostes', 'fui', 'geral', 'grande', 'grandes', 'grupo', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'la', 'lá', 'lado', 'lhe', 'lhes', 'lo', 'local', 'logo', 'longe', 'lugar', 'maior', 'maioria', 'mais', 'mal', 'mas', 'máximo', 'me', 'meio', 'menor', 'menos', 'mês', 'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'mil', 'minha', 'minhas', 'momento', 'muita', 'muitas', 'muito', 'muitos', 'na', 'nada', 'não', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'nem', 'nenhum', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 'ninguém', 'nível', 'no', 'noite', 'nome', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 'número', 'nunca', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'parece', 'parte', 'partir', 'paucas', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'pude', 'pôde', 'podem', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'põe', 'põem', 'pois', 'ponto', 'pontos', 'por', 'porém', 'porque', 'porquê', 'posição', 'possível', 'possivelmente', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'puderam', 'quais', 'quáis', 'qual', 'quando', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quê', 'quem', 'quer', 'quereis', 'querem', 'queremas', 'queres', 'quero', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'são', 'se', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sejamos', 'sem', 'sempre', 'sendo', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'sétima', 'sétimo', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'só', 'sob', 'sobre', 'sois', 'somos', 'sou', 'sua', 'suas', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tanto', 'tão', 'tarde', 'te', 'tem', 'tém', 'têm', 'temos', 'tendes', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'ti', 'tido', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes', 'toda', 'todas', 'todavia', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas', 'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'vai', 'vais', 'vão', 'vários', 'vem', 'vêm', 'vendo', 'vens', 'ver', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'vir', 'você', 'vocês', 'vos', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '_' ]
mais_stopwords2 = str(mais_stopwords1).strip('[]')
mais_stopwords3 = mais_stopwords + mais_stopwords2
mais_stopwords3 = unidecode.unidecode(mais_stopwords3) 
lista_mais_stopwords = mais_stopwords3.split(', ')
lista_final_stopwords = advanced_sw_list1 + lista_mais_stopwords

In [ ]:
#Definindo idioma para recuperação das stopwords
stops = stopwords.words('portuguese')

In [ ]:
# Transformando punctuation em uma lista, e inserindo na lista das stops_words
punctuation = list(punctuation)
# Juntando as duas listas em uma só
stops.extend(punctuation)

In [ ]:
#Colocando título como substituto do texto
contador = 0
indices=[]
for row in df['message_norm_treatment_ssw2']:
  if(len(row)==2):
    indices.append(contador)
    df['message_norm_treatment_ssw2'][contador] = df['title_norm'][contador]
  contador+=1

In [ ]:
#Tokenizando coluna para remover as stopwords
contador3 = 0
for row in df['message_norm_treatment_ssw2']:
  if(contador3 in indices):
    df['message_norm_treatment_ssw2'][contador3] = word_tokenize( df['message_norm_treatment_ssw2'][contador3])
    print(df['message_norm_treatment_ssw2'][contador3])
  contador3+=1

['simpsons', 'tera', 'episodio', 'sobre', 'manifestacoes', 'no', 'brasil']
['lulinha', ',', 'filho', 'de', 'lula', ',', 'comprou', 'aviao', 'gulfstream', 'por', '100', 'milhoes', 'de', 'reais']
['pastor', 'sergio', 'von', 'helder', ',', 'que', 'chutou', 'santa', ',', 'nao', 'virou', 'catolico']
['mensagem', 'falsa', 'conta', 'briga', 'de', 'luiz', 'claudio', 'lula', 'da', 'silva', 'no', 'cirque', 'du', 'soleil']
['vestidinho', 'branco', '-', 'historia', 'de', 'menina', 'que', 'aceitou', 'jesus', 'e', 'foi', 'morta', 'pelo', 'pai', 'e', 'ficticia']
['natanael', 'bufalo', ',', 'que', 'matou', 'menina', 'de', '10', 'anos', ',', 'nao', 'esta', 'foragido']
['mentira', '-', 'chineses', 'comem', 'bebes', 'para', 'aumentar', 'desempenho']
['virus', ':', 'homem', 'ao', 'fazer', 'teste', 'de', 'fidelidade', 'descobre', 'que', 'mulher', 'e', 'lesbica']
['mensagem', 'falsa', '-', 'rato', 'e', 'encontrado', 'no', 'acougue', 'do', 'supermercado', 'extra']
['senha', 'invertida', 'avisa', 'policia', '

In [ ]:
#Removendo as stopwords na coluna que tem a notícia tokenizada
contador4 = 0
for row in df['message_norm_treatment_ssw2']:
  if(contador4 in indices):
    df['message_norm_treatment_ssw2'][contador4] = [word for word in row if word not in stops]
  contador4+=1

In [ ]:
#Removendo as stopwords avançadas na coluna que tem a notícia tokenizada
contador = 0
for row in df['message_norm_treatment_ssw2']:
  if(contador in indices):
    df['message_norm_treatment_ssw2'][contador] = [word for word in row if ((word not in lista_final_stopwords) and (len(word)>3))]
  contador+=1

In [ ]:
df['message_norm_treatment_ssw2']

0             [simpsons, episodio, manifestacoes, brasil]
1       ['alerta', 'geral', 'atencao', 'fanta', 'propa...
2       ['novo', 'mega', 'campeao', 'brasil', 'enrique...
3       [lulinha, filho, lula, comprou, aviao, gulfstr...
4       [pastor, sergio, helder, chutou, santa, virou,...
                              ...                        
5196                                            [contato]
5197                                            [autores]
5198                                           [pesquise]
5199                              [politica, privacidade]
5200    [assai, atacadista, doando, alcool, cestas, ba...
Name: message_norm_treatment_ssw2, Length: 5201, dtype: object

In [ ]:
transforma_maiusculas_para_minusculas = lambda x: str(x).lower()
# Aplica a função lambda em todos os elementos do dataframe
df['message_norm_treatment_ssw2'] = df['message_norm_treatment_ssw2'].apply(transforma_maiusculas_para_minusculas)

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df_antes_2016 = df[df["datetime"].dt.year < 2016]

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df_2016 = df[df["datetime"].dt.year == 2016]

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df_2017 = df[df["datetime"].dt.year == 2017]

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df_2018 = df[df["datetime"].dt.year == 2018]

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df_2019 = df[df["datetime"].dt.year == 2019]

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df_2020 = df[df["datetime"].dt.year == 2020]

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df_2021 = df[df["datetime"].dt.year == 2021]

## SEPARAÇÃO TEMPORAL

### 2013-2015

In [ ]:
trienio = df_antes_2016['message_norm_treatment_ssw2']

In [ ]:
trienio = trienio.apply(lambda x: ' '.join(x))

In [ ]:
trienio

0      [ ' s i m p s o n s ' ,   ' e p i s o d i o ' ...
1      [ ' a l e r t a ' ,   ' g e r a l ' ,   ' a t ...
2      [ ' n o v o ' ,   ' m e g a ' ,   ' c a m p e ...
3      [ ' l u l i n h a ' ,   ' f i l h o ' ,   ' l ...
4      [ ' p a s t o r ' ,   ' s e r g i o ' ,   ' h ...
                             ...                        
215    [ ' v e r d a d e ' ,   ' p o b r e ' ,   ' e ...
216    [ ' n o i t e ' ,   ' p e s s o a l ' ,   ' f ...
217    [ ' c r i s t i a n i s m o ' ,   ' b a n i d ...
218    [ ' p a p a i ' ,   ' n o e l ' ,   ' i n v e ...
219    [ ' s a i b a ' ,   ' d i a s ' ,   ' 2 0 1 5 ...
Name: message_norm_treatment_ssw2, Length: 220, dtype: object

### 2016

In [ ]:
tm_2016 = df_2016['message_norm_treatment_ssw2']

In [ ]:
tm_2016 = tm_2016.apply(lambda x: ' '.join(x))

In [ ]:
tm_2016

220    [ ' b a t e u ' ,   ' f i l h a ' ,   ' a n o ...
221    [ ' f i l h a ' ,   ' r e n a n ' ,   ' c a l ...
222    [ ' m a m a e ' ,   ' a t e n c a o ' ,   ' s ...
223    [ ' u r g e n t e ' ,   ' d i g a ' ,   ' c o ...
224    [ ' v e r d a d e ' ,   ' p r e - s a l ' ,   ...
                             ...                        
787    [ ' v i d e o ' ,   ' f l a g r a ' ,   ' s u ...
788    [ ' a d o r e i ' ,   ' n o v a ' ,   ' f u n ...
789    [ ' l u t e i ' ,   ' d i t a d u r a ' ,   ' ...
790    [ ' m a r c o s ' ,   ' t r i g u e i r o ' , ...
791    [ ' d o a c a o ' ,   ' f i l h o t e s ' ,   ...
Name: message_norm_treatment_ssw2, Length: 572, dtype: object

### 2017

In [ ]:
tm_2017 = df_2017['message_norm_treatment_ssw2']

In [ ]:
tm_2017 = tm_2017.apply(lambda x: ' '.join(x))

In [ ]:
tm_2017

792     [ ' m a d r u g a d a ' ,   ' f a l l e c i o ...
793     [ ' p i t b u l l ' ,   ' a t a c a ' ,   ' l ...
794     [ ' m o t o r i s t a ' ,   ' u b e r ' ,   ' ...
795     [ ' 2 0 1 7 ' ,   ' m a g i c o . 0 1 - 0 1 - ...
796     [ ' f a m i l i a ' ,   ' o b a m a ' ,   ' d ...
                              ...                        
1526    [ ' s h a k e s p e a r e ' ,   ' s i n t o ' ...
1527    [ ' c a r t a o ' ,   ' c r e d i t o ' ,   ' ...
1528    [ ' c a r l o s ' ,   ' d i r e t o r ' ,   ' ...
1529    [ ' p o l i c i a ' ,   ' f e d e r a l ' ,   ...
1530    [ ' t h o m a s ' ,   ' b e a t i e ' ,   ' r ...
Name: message_norm_treatment_ssw2, Length: 739, dtype: object

### 2018

In [ ]:
tm_2018 = df_2018['message_norm_treatment_ssw2']

In [ ]:
tm_2018 = tm_2018.apply(lambda x: ' '.join(x))

In [ ]:
tm_2018

1531    [ ' d o r i a ' ,   ' c a n c e l a ' ,   ' h ...
1532    [ ' i r m a o ' ,   ' l u l a ' ,   ' r e v e ...
1533    [ ' b u r g e r ' ,   ' k i n g ' ,   ' d a n ...
1534    [ ' v i r a d a ' ,   ' t i j u c a ' ,   ' q ...
1535    [ ' a s s a i ' ,   ' a t a c a d i s t a ' , ...
                              ...                        
2560    [ ' i n s c r i c o e s ' ,   ' p r o j e t o ...
2561    [ ' c o m p r e i ' ,   ' m e l a n c i a ' , ...
2562    [ ' n e t o ' ,   ' r a u l ' ,   ' s e i x a ...
2563    [ ' a n o s ' ,   ' f a b i o ' ,   ' j u n i ...
2564    [ ' g o v e r n o ' ,   ' g r o e l a n d i a ...
Name: message_norm_treatment_ssw2, Length: 1034, dtype: object

### 2019

In [ ]:
tm_2019 = df_2019['message_norm_treatment_ssw2']

In [ ]:
tm_2019 = tm_2019.apply(lambda x: ' '.join(x))

In [ ]:
tm_2019

2565    [ ' b o l s o n a r o ' ,   ' p e d e ' ,   ' ...
2566    [ ' m i c h e l l e ' ,   ' b o l s o n a r o ...
2567    [ ' t a r d e ' ,   ' p e s s o a l ' ,   ' i ...
2568    [ ' a c o r d a r ' ,   ' a m e r i c a ' ,   ...
2569    [ ' s i t e ' ,   ' s e g u r a d o r a ' ,   ...
                              ...                        
3438    [ ' r e d e ' ,   ' g l o b o ' ,   ' c o n t ...
3439    [ ' j o s e ' ,   ' a b r e u ' ,   ' p e g o ...
3440    [ ' v e r d a d e i r o ' ,   ' m a n e * f l ...
3441    [ ' v e j a ' ,   ' r e a l ' ,   ' m o t i v ...
3442    [ ' c i d a d a o ' ,   ' f a b i o ' ,   ' p ...
Name: message_norm_treatment_ssw2, Length: 878, dtype: object

### 2020

In [ ]:
tm_2020 = df_2020['message_norm_treatment_ssw2']

In [ ]:
tm_2020 = tm_2020.apply(lambda x: ' '.join(x))

In [ ]:
tm_2020

3443    [ ' l u l i n h a ' ,   ' c p f s ' ,   ' c a ...
3444    [ ' g o v e r n o ' ,   ' f e d e r a l ' ,   ...
3445    [ ' a p o i a r ' ,   ' b o l s o n a r o ' , ...
3446    [ ' c a r r o ' ,   ' m o v i d o ' ,   ' a g ...
3447    [ ' r e s p o n d a ' ,   ' p e r g u n t a s ...
                              ...                        
4600    [ ' h o j e ' ,   ' j a p a o ' ,   ' f o r m ...
4601    [ ' m a i o r ' ,   ' f r a u d e ' ,   ' h u ...
4602    [ ' h o m e m ' ,   ' m o r r e u ' ,   ' a t ...
4603    [ ' j o r n a l i s t a ' ,   ' f i c a d o ' ...
4604    [ ' a l e r t a r ' ,   ' r e s e t ' ,   ' m ...
Name: message_norm_treatment_ssw2, Length: 1162, dtype: object

### 2021

In [ ]:
tm_2021 = df_2021['message_norm_treatment_ssw2']

In [ ]:
tm_2021 = tm_2021.apply(lambda x: ' '.join(x))

In [ ]:
tm_2021

4605    [ ' m a r i o ' ,   ' s e r g i o ' ,   ' c o ...
4606    [ ' v a c i n a ' ,   ' p f i z e r ' ,   ' a ...
4607    [ ' d e n u n c i a ' ,   ' g r a v i s s i m ...
4608    [ ' c a r a ' ,   ' p a r i s ' ,   ' e n c h ...
4609    [ ' a t e n c a o ' ,   ' b r a s i l ' ,   ' ...
                              ...                        
5191    [ ' b r a s i l ' ,   ' e x p u l s o ' ,   ' ...
5192    [ ' d i l u v i o ' ,   ' c h i n a ' ,   ' m ...
5193    [ ' m a r i a n a ' ,   ' g o d o y ' ,   ' g ...
5194    [ ' m i l i t a r ' ,   ' p r e n d e ' ,   ' ...
5195    [ ' g e n t e ' ,   ' v e l h o ' ,   ' l u l ...
Name: message_norm_treatment_ssw2, Length: 591, dtype: object

## SEPARAÇÃO CATEGÓRICA

### Política

In [ ]:
aux = df.loc[df.category == 'Política']
tm_politica = aux['message_norm_treatment_ssw2']

In [ ]:
tm_politica = tm_politica.apply(lambda x: ''.join(x))

In [ ]:
tm_politica

2       ['novo', 'mega', 'campeao', 'brasil', 'enrique...
3       ['lulinha', 'filho', 'lula', 'comprou', 'aviao...
5       ['manda', 'nesse', 'pais', 'sento', 'quiser', ...
6       ['mensagem', 'falsa', 'conta', 'briga', 'luiz'...
20      ['foto', 'dilma', 'rousseff', 'fuzil', 'montag...
                              ...                        
5176    ['estadao', 'exclusivo', 'farsa', 'revelada', ...
5179    ['urgente', 'exercito', 'determina', 'voto', '...
5189    ['vaza', 'video', 'joice', 'hasselmann', 'mach...
5190    ['bolsonaro', 'casamento', 'isolada', 'ex-mulh...
5195    ['gente', 'velho', 'luladrao', 'endoideceu', '...
Name: message_norm_treatment_ssw2, Length: 1508, dtype: object

### Brasil

In [ ]:
aux = df.loc[df.category == 'Brasil']
tm_brasil = aux['message_norm_treatment_ssw2']

In [ ]:
tm_brasil = tm_brasil.apply(lambda x: ''.join(x))

In [ ]:
tm_brasil

8       ['natanael', 'bufalo', 'matou', 'menina', 'ano...
13      ['senha', 'invertida', 'avisa', 'policia', 'ca...
19      ['explica', 'video', 'ovni', 'agudos-sp', 'par...
21      ['royal', 'canin', 'dona', 'instituto', 'royal...
27      ['ashley', 'flores', 'desaparecida', 'duas', '...
                              ...                        
5178    ['foto', 'mostra', 'rodovia', 'asfaltada', 'sp...
5183    ['olha', 'frio', 'chegando', 'parana', 'chuva'...
5185    ['produtores', 'tomam', 'medidas', 'evitar', '...
5186    ['neve', 'curitiba', 'gelo', 'cobrir', 'pista'...
5193    ['mariana', 'godoy', 'globo', 'redetv', 'suica...
Name: message_norm_treatment_ssw2, Length: 1013, dtype: object

### Saúde

In [ ]:
aux = df.loc[df.category == 'Saúde']
tm_saude = aux['message_norm_treatment_ssw2']

In [ ]:
tm_saude = tm_saude.apply(lambda x: ''.join(x))

In [ ]:
tm_saude

1       ['alerta', 'geral', 'atencao', 'fanta', 'propa...
10      ['bomba', 'saiba', 'querer', 'comer', 'gosta',...
12      ['mensagem', 'falsa', 'rato', 'encontrado', 'a...
14                            ['*agua', 'dormir**.cerca']
16      ['inca', 'precisa', 'doacoes', 'toucas', 'cria...
                              ...                        
5151    ['gente', 'reclassificados', 'olhem', 'legal',...
5163    ['policia', 'federal', 'desenterra', 'caixoes'...
5166    ['linda', 'homenagem', 'hospital', 'lucas', 'c...
5171    ['informacao', 'infectado', 'covid', 'vezes', ...
5182    ['hospital', 'lucas-puc', 'amanhaceu', 'pacien...
Name: message_norm_treatment_ssw2, Length: 673, dtype: object

### Mundo

In [ ]:
aux = df.loc[df.category == 'Mundo']
tm_mundo = aux['message_norm_treatment_ssw2']

In [ ]:
tm_mundo = tm_mundo.apply(lambda x: ''.join(x))

In [ ]:
tm_mundo

9       ['mentira', 'chineses', 'comem', 'bebes', 'aum...
15      ['atencao', 'garotas', 'filipinas', 'pisoteara...
18      ['homem', 'filmado', 'agredindo', 'menino', 'a...
22      ['video', 'carro', 'salva', 'aviao', 'tragedia...
28      ['criatura', 'estranha', 'escala', 'predio', '...
                              ...                        
5170    ['cubanos', 'jogam', 'artistas', 'jornalistas'...
5174    ['argentina', 'normal', 'argentina', 'fechado'...
5181    ['fazendeiros', 'franceses', 'capricharam', 'p...
5192    ['diluvio', 'china', 'matou', 'pessoas', 'tune...
5194    ['militar', 'prende', 'bill', 'gates', 'terca-...
Name: message_norm_treatment_ssw2, Length: 530, dtype: object

### Ciência

In [ ]:
aux = df.loc[df.category == 'Ciência']
tm_ciencia = aux['message_norm_treatment_ssw2']

In [ ]:
tm_ciencia = tm_ciencia.apply(lambda x: ''.join(x))

In [ ]:
tm_ciencia

51      ['teoria', 'falsa', 'masaru', 'emoto', 'palavr...
63      ['ilusao', 'otica', 'carro', 'fantasma', 'apar...
98      ['mito', 'pessoas', 'letra', 'feia', 'intelige...
114     ['tempo', 'ouvido', 'falar', 'famoso', 'bio-ch...
122     ['ilusao', 'otica', 'foto', 'vestido', 'branco...
                              ...                        
4699    ['*atencao*', 'fenomeno', 'equinocio', 'afetar...
4770    ['imagens', 'marte', 'durante', 'noite', 'envi...
4945                         ['ovnis', 'jaragua', 'sul/']
5021    ['imagine', 'sentar-se', 'neste', 'lugar', 'du...
5175    ['sabe', 'ensinar', 'precisa', 'saber', 'situa...
Name: message_norm_treatment_ssw2, Length: 73, dtype: object

### Esporte

In [ ]:
aux = df.loc[df.category == 'Esporte']
tm_esporte = aux['message_norm_treatment_ssw2']

In [ ]:
tm_esporte = tm_esporte.apply(lambda x: ''.join(x))

In [ ]:
tm_esporte

38      ['virus', 'logotipo', 'copa', 'mundo', 'mensag...
40      ['copa', 'possibilidade', 'copa', 'brasil', 'c...
42      ['materia', 'falsa', 'france', 'football', 'co...
60      ['lista', 'falsa', 'convocados', 'brasil', 'co...
72      ['selecao', 'japonesa', 'viajou', 'copa', 'mun...
                              ...                        
5153    ['declaracao', 'tanta', 'perna', 'gente', 'rui...
5154    ['futebol', 'atacante', 'bahia', 'assume', 'ho...
5187    ['italo', 'ferreira', 'maior', 'premio', 'lula...
5188               ['custo', 'vila', 'olimpica', 'japao']
5191    ['brasil', 'expulso', 'olimpiadas', 'causa', '...
Name: message_norm_treatment_ssw2, Length: 217, dtype: object

### Religião

In [ ]:
aux = df.loc[df.category == 'Religião']
tm_religiao = aux['message_norm_treatment_ssw2']

In [ ]:
tm_religiao = tm_religiao.apply(lambda x: ''.join(x))

In [ ]:
tm_religiao

4       ['pastor', 'sergio', 'helder', 'chutou', 'sant...
7       ['vestidinho', 'branco', 'historia', 'menina',...
24      ['ficcao', 'jovem', 'humilhado', 'cristao', 'j...
25      ['conforme', 'publicacao', 'jornal', 'portugue...
29      ['montagem', 'serpente', 'sete', 'cabecas', 'e...
                              ...                        
4887    ['irmaos', 'passando', 'grupos', 'pedido', 'ur...
4979    ['letra', 'musica', 'psicografia', 'chico', 'x...
5041    ['brasil', 'inteiro', 'corpus', 'christi', 'po...
5057    ['noticia_urgente', 'pastor', 'soares', 'funda...
5161    ['janeiro', 'estado', 'critico', 'olha', 'most...
Name: message_norm_treatment_ssw2, Length: 245, dtype: object

### Entretenimento

In [ ]:
aux = df.loc[df.category == 'Entretenimento']
tm_entretenimento = aux['message_norm_treatment_ssw2']

In [ ]:
tm_entretenimento = tm_entretenimento.apply(lambda x: ''.join(x))

In [ ]:
tm_entretenimento

0       ['simpsons', 'episodio', 'manifestacoes', 'bra...
35      ['avril', 'lavigne', 'morreu', 'substituida', ...
47      ['foto', 'justin', 'bieber', 'beijando', 'home...
55      ['michael', 'jackson', 'vivo', 'teoria', 'cant...
78      ['ator', 'sacha', 'baron', 'derruba', 'atriz',...
                              ...                        
5039    ['*filme', 'intitulado*', '*sorry*', 'dura', '...
5108    ['fatima', 'bernardes', 'questiona', 'excesso'...
5140    ['gabriel', 'garcia', 'marquez', 'retirou', 'v...
5145    ['inacreditavel', 'coral', 'francisco', 'conve...
5180    ['parte', 'quadro', 'funcionarios', 'globo', '...
Name: message_norm_treatment_ssw2, Length: 431, dtype: object

### Tecnologia

In [ ]:
aux = df.loc[df.category == 'Tecnologia']
tm_tecnologia = aux['message_norm_treatment_ssw2']

In [ ]:
tm_tecnologia = tm_tecnologia.apply(lambda x: ''.join(x))

In [ ]:
tm_tecnologia

11      ['virus', 'homem', 'teste', 'fidelidade', 'des...
17      ['facebook', 'doara', 'compartilhamento', 'tra...
31      ['ultra', 'slim', 'virus', 'facebook', 'produt...
32      ['virus', 'mulher', 'descobre', 'traicao', 'ma...
34      ['numero', 'celular', 'clona', 'numero', 'repa...
                              ...                        
5136    ['auxilio', 'cesta', 'basica', 'beneficio', 'a...
5137    ['mercado', 'livre', 'parabens', 'promocao', '...
5160    ['vivo', 'informa', 'parabens', 'ganhar', 'sup...
5172    ['americanas', 'comemoracao', 'aniversario', '...
5184    ['apagao', 'cnpq', 'resultou', 'perda', 'dados...
Name: message_norm_treatment_ssw2, Length: 480, dtype: object

## SEPARAÇÃO GERAL

In [ ]:
tm_geral = df['message_norm_treatment_ssw2']

In [ ]:
tm_geral = tm_geral.apply(lambda x: ''.join(x))

In [ ]:
tm_geral

0       ['simpsons', 'episodio', 'manifestacoes', 'bra...
1       ['alerta', 'geral', 'atencao', 'fanta', 'propa...
2       ['novo', 'mega', 'campeao', 'brasil', 'enrique...
3       ['lulinha', 'filho', 'lula', 'comprou', 'aviao...
4       ['pastor', 'sergio', 'helder', 'chutou', 'sant...
                              ...                        
5196                                          ['contato']
5197                                          ['autores']
5198                                         ['pesquise']
5199                          ['politica', 'privacidade']
5200    ['assai', 'atacadista', 'doando', 'alcool', 'c...
Name: message_norm_treatment_ssw2, Length: 5201, dtype: object

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

## Método LSA - APLICAÇÃO NO DATASET DO TRIÊNIO

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True Remove pontuações
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = df_antes_2016.message_norm_treatment_ssw2.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['simpsons', 'episodio', 'manifestacoes', 'brasil']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['simpsons', 'episodio', 'Manifestacoes']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.5675587871756065
Coherence score with 5 clusters: 0.4687616570007854
Coherence score with 6 clusters: 0.6105897752849471
Coherence score with 7 clusters: 0.6384412013605857
Coherence score with 8 clusters: 0.6304464129527062
Coherence score with 9 clusters: 0.6329607003224733
Coherence score with 10 clusters: 0.6807247404380237
Coherence score with 11 clusters: 0.5906255630994702
Coherence score with 12 clusters: 0.5662916701540214
Coherence score with 13 clusters: 0.5910384701656697
Coherence score with 14 clusters: 0.627687239495169
Coherence score with 15 clusters: 0.6260988241708346
Coherence score with 16 clusters: 0.5873204935594551
Coherence score with 17 clusters: 0.6791579282176348
Coherence score with 18 clusters: 0.5634694067011867
Coherence score with 19 clusters: 0.5312251289433598
Coherence score with 20 clusters: 0.5989581084103374


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=10, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.309*"fazenda" + 0.258*"querer" + 0.199*"falar" + 0.161*"animal" + 0.153*"pessoa" + 0.146*"tomar" + 0.132*"vaca" + 0.132*"saber" + 0.131*"presidente" + 0.119*"lugar".
Words in 1: -0.408*"fazenda" + 0.278*"querer" + -0.212*"animal" + 0.197*"falar" + -0.181*"vaca" + 0.155*"lugar" + 0.129*"mandar" + -0.122*"possivel" + 0.098*"Filho" + 0.094*"dizer".
Words in 2: 0.398*"cama" + 0.309*"gazin" + 0.221*"saquinho" + 0.214*"terra" + 0.205*"comprar" + 0.177*"cemiterio" + 0.137*"video" + 0.137*"produto" + 0.135*"dormir" + 0.130*"tirar".
Words in 3: -0.330*"atleta" + -0.283*"gesto" + -0.211*"presidente" + -0.181*"brasileiro" + -0.141*"proibir" + -0.140*"imprensar" + -0.138*"respeito" + -0.123*"berro" + 0.108*"lugar" + -0.098*"manifestar".
Words in 4: -0.358*"pessoa" + -0.128*"passar" + 0.120*"cama" + -0.118*"tomar" + -0.112*"enviar" + -0.110*"problema" + -0.107*"pesquisador" + 0.107*"fazenda" + -0.106*"doenca" + 0.103*"comprar".
Words in 5: -0.341*"enviar" + -0.209*"crime" + -0.161*"re

## Método LSA - APLICAÇÃO NO DATASET DE 2016

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = df_2016.message_norm_treatment_ssw2.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['bateu', 'filha', 'anos', 'morto', 'cadeia']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['bater', 'filha', 'ano', 'matar', 'cadeia']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.3601223585716831
Coherence score with 5 clusters: 0.4200098413182922
Coherence score with 6 clusters: 0.3783588215575953
Coherence score with 7 clusters: 0.48148869251048076
Coherence score with 8 clusters: 0.3393936359755879
Coherence score with 9 clusters: 0.36311351220787974
Coherence score with 10 clusters: 0.4367344018542312
Coherence score with 11 clusters: 0.439042625106601
Coherence score with 12 clusters: 0.40893412039043375
Coherence score with 13 clusters: 0.38847866593168623
Coherence score with 14 clusters: 0.37593353162808535
Coherence score with 15 clusters: 0.3805879368523829
Coherence score with 16 clusters: 0.3929771625413423
Coherence score with 17 clusters: 0.3788098071174185
Coherence score with 18 clusters: 0.35790696529039373
Coherence score with 19 clusters: 0.39770318965462736
Coherence score with 20 clusters: 0.3149658127579696


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=7, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: -0.227*"pessoa" + -0.183*"falar" + -0.166*"ficar" + -0.147*"gente" + -0.138*"ano" + -0.132*"passar" + -0.129*"saber" + -0.120*"caso" + -0.119*"governo" + -0.115*"brasileiro".
Words in 1: 0.266*"falar" + -0.223*"causar" + 0.180*"filha" + -0.176*"quimico" + 0.175*"senhor" + -0.163*"microcefalia" + -0.161*"produto" + -0.145*"ministerio_saude" + -0.143*"vacina" + 0.142*"gente".
Words in 2: -0.315*"falar" + -0.253*"filha" + -0.238*"senhor" + -0.179*"caso" + -0.142*"shopping" + 0.133*"governo" + -0.125*"causar" + -0.118*"forte" + -0.114*"estrutura" + -0.113*"juntar".
Words in 3: -0.385*"mulher" + -0.171*"caso" + -0.156*"direito" + -0.150*"denunciar" + -0.148*"pico_maximo" + 0.147*"quimico" + -0.146*"vacino" + 0.138*"produto" + -0.129*"explicar" + -0.128*"provocar".
Words in 4: -0.227*"governo" + -0.210*"brasileiro" + 0.181*"mulher" + -0.154*"congresso" + -0.144*"boliviano" + 0.140*"ficar" + 0.140*"crianca" + 0.135*"gente" + -0.132*"empresa" + -0.129*"presidente".
Words in 5: -0.3

## Método LSA - APLICAÇÃO NO DATASET DE 2017

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = df_2017.message_norm_treatment_ssw2.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['madrugada', 'fallecio', 'cesar', 'millan', 'encantador', 'encuentra', 'mundo', 'tras', 'noticia', 'dejado', 'paralizados', 'manana', 'quedo', 'oficialmente', 'confirmada', 'diversos', 'medios', 'comunicacion', 'muerte', 'famoso', 'encantador', 'perros', 'cesar', 'millan', 'quien', 'fallecio', 'madrugada', 'ayer', 'tras', 'sufrir']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['madrugada', 'encantador', 'encuentrar', 'Mundo', 'tras', 'noticiar', 'dejar', 'paralizar', 'Manana', 'quedo', 'oficialmente', 'confirmar', 'medio', 'muerte', 'famoso', 'encantador', 'perro', 'madrugar', 'ayer', 'tras', 'sufrir', 'fulminate', 'paro', 'cardiaco', 'interior', 'domicilio', 'ano', 'quien', 'hiciera', 'famoso']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.5397637210449002
Coherence score with 5 clusters: 0.4012392169427711
Coherence score with 6 clusters: 0.5252097662844052
Coherence score with 7 clusters: 0.4332926207849132
Coherence score with 8 clusters: 0.41760680438983144
Coherence score with 9 clusters: 0.5237880127100976
Coherence score with 10 clusters: 0.4563213029734256
Coherence score with 11 clusters: 0.4288841113444784
Coherence score with 12 clusters: 0.40869573426364925
Coherence score with 13 clusters: 0.4869226221989567
Coherence score with 14 clusters: 0.4646666380898606
Coherence score with 15 clusters: 0.3662567444314577
Coherence score with 16 clusters: 0.43895856360998364
Coherence score with 17 clusters: 0.39305509068286704
Coherence score with 18 clusters: 0.35753972153295804
Coherence score with 19 clusters: 0.453041078132057
Coherence score with 20 clusters: 0.4271790843661892


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=4, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: -0.281*"falar" + -0.243*"pessoa" + -0.180*"saber" + -0.158*"mensagem" + -0.129*"acontecer" + -0.123*"gente" + -0.122*"passar" + -0.121*"ano" + -0.119*"querer" + -0.113*"entrar".
Words in 1: -0.317*"globo" + -0.267*"temer" + -0.250*"jornalismo" + 0.222*"mensagem" + -0.211*"rede_globo" + -0.209*"alinhar" + -0.206*"noticia" + 0.140*"contato" + -0.126*"jornal" + -0.125*"vincular".
Words in 2: -0.371*"desconto" + -0.343*"aereo" + -0.341*"passagem" + -0.284*"idoso" + -0.278*"mensagem" + -0.233*"contato" + -0.189*"empresa" + 0.118*"falar" + 0.103*"gente" + -0.099*"grande".
Words in 3: 0.616*"fruta" + 0.349*"comer" + 0.218*"estomago" + 0.200*"mensagem" + 0.150*"comida" + 0.148*"paciente" + 0.142*"vazio" + 0.141*"cancro" + 0.127*"forma" + 0.105*"fatia".


## Método LSA - APLICAÇÃO NO DATASET DE 2018

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = df_2018.message_norm_treatment_ssw2.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['doria', 'cancela', 'homenagem', 'dona', 'marisa', 'viaduto', 'nome', 'professora', 'heley']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['dorio', 'cancelar', 'homenagem', 'viaduto', 'nome', 'professora']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.6170136460648865
Coherence score with 5 clusters: 0.5306272292926292
Coherence score with 6 clusters: 0.55104340074215
Coherence score with 7 clusters: 0.5793893917360678
Coherence score with 8 clusters: 0.575312105362538
Coherence score with 9 clusters: 0.5403506208723301
Coherence score with 10 clusters: 0.5167271927556939
Coherence score with 11 clusters: 0.4654535988429147
Coherence score with 12 clusters: 0.5218433268937495
Coherence score with 13 clusters: 0.5596056810505786
Coherence score with 14 clusters: 0.4982495892087561
Coherence score with 15 clusters: 0.4747003951664345
Coherence score with 16 clusters: 0.47545036075581304
Coherence score with 17 clusters: 0.46875204311967483
Coherence score with 18 clusters: 0.4436449975136594
Coherence score with 19 clusters: 0.4431174842508744
Coherence score with 20 clusters: 0.46166792991221184


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=4, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.234*"bolsonaro" + 0.209*"pessoa" + 0.179*"ficar" + 0.158*"querer" + 0.155*"passar" + 0.146*"saber" + 0.144*"ano" + 0.133*"falar" + 0.127*"gente" + 0.122*"presidente".
Words in 1: 0.329*"forca" + 0.251*"area" + 0.193*"auxiliar" + 0.176*"delegar" + 0.153*"militar" + 0.151*"intervencao_militar" + 0.150*"policia" + 0.148*"quartel" + 0.144*"estadual" + 0.144*"especial".
Words in 2: -0.660*"bolsonaro" + -0.195*"candidato" + -0.160*"voto" + 0.152*"passar" + 0.137*"ajudar" + -0.120*"votar" + -0.106*"presidente" + -0.105*"eleicao" + -0.104*"urna" + 0.100*"mensagem".
Words in 3: -0.298*"olhar" + 0.270*"gripe" + -0.211*"comer" + -0.183*"passar" + 0.181*"vacino" + 0.178*"paciente" + 0.172*"chegar" + 0.171*"ano" + 0.163*"tomar" + 0.146*"falar".


## Método LSA - APLICAÇÃO NO DATASET DE 2019

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = df_2019.message_norm_treatment_ssw2.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['bolsonaro', 'pede', 'eleitores', 'ignorem', 'partidos', 'vermelhos', 'anti', 'brasileiros']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['bolsonaro', 'pedir', 'eleitor', 'ignorar', 'partido', 'vermelho', 'anti', 'brasileiro']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.4893616792842391
Coherence score with 5 clusters: 0.4306672809253927
Coherence score with 6 clusters: 0.4506440865643499
Coherence score with 7 clusters: 0.4025194802910517
Coherence score with 8 clusters: 0.42321495100192497
Coherence score with 9 clusters: 0.45382509691904194
Coherence score with 10 clusters: 0.37477498194245673
Coherence score with 11 clusters: 0.4393273895794817
Coherence score with 12 clusters: 0.3731483057181304
Coherence score with 13 clusters: 0.3823829570799054
Coherence score with 14 clusters: 0.4163257680291113
Coherence score with 15 clusters: 0.4190800365043979
Coherence score with 16 clusters: 0.3879640771296204
Coherence score with 17 clusters: 0.4038570532141187
Coherence score with 18 clusters: 0.3951694144545029
Coherence score with 19 clusters: 0.3689787193107883
Coherence score with 20 clusters: 0.378096246309253


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=4, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: -0.346*"acabar" + -0.263*"pessoa" + -0.180*"ano" + -0.179*"falar" + -0.169*"presidente" + -0.155*"bolsonaro" + -0.144*"aumento" + -0.144*"salario" + -0.144*"partido" + -0.141*"ficar".
Words in 1: 0.475*"acabar" + 0.247*"aumento" + -0.214*"presidente" + 0.211*"salario" + 0.208*"partido" + -0.195*"falar" + 0.189*"aprovar" + 0.138*"redutor" + 0.138*"aposentado" + 0.135*"reduzir".
Words in 2: -0.519*"presidente" + -0.356*"bolsonaro" + 0.168*"abaixo" + 0.162*"amigo" + 0.159*"site" + 0.153*"pessoa" + -0.149*"brasileiro" + 0.126*"compartilhar" + 0.121*"receber" + 0.120*"casa".
Words in 3: -0.326*"abaixo" + -0.323*"site" + 0.311*"falar" + 0.237*"pessoa" + -0.208*"receber" + -0.197*"compartilhar" + -0.150*"presidente" + 0.128*"matar" + -0.105*"ganhar" + -0.099*"mensagem".


## Método LSA - APLICAÇÃO NO DATASET DE 2020

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = df_2020.message_norm_treatment_ssw2.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['lulinha', 'cpfs', 'cadeia', 'nesse', 'filho', 'puta', 'puta', 'parte', 'receita', 'federal', 'descobre', 'lulinha', 'alterava', 'data', 'nascimento', 'filiacao', 'receita', 'federal', 'descobre', 'lulinha', 'dono', 'alterava', 'filiacao', 'data', 'nascimento']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['lulinha', 'cpfs', 'cadeia', 'filho', 'puto', 'puto', 'parte', 'receita', 'federal', 'descobrir', 'lulinha', 'alterar', 'data', 'nascimento', 'filiacao', 'receita', 'federal', 'descobrir', 'lulinha', 'dono', 'alterar', 'filiacao', 'data', 'nascimento']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.3932720862521225
Coherence score with 5 clusters: 0.4278598939116004
Coherence score with 6 clusters: 0.49901715888414727
Coherence score with 7 clusters: 0.33859924761402904
Coherence score with 8 clusters: 0.3700880854861172
Coherence score with 9 clusters: 0.33072074144711594
Coherence score with 10 clusters: 0.4554863767483379
Coherence score with 11 clusters: 0.391828953323175
Coherence score with 12 clusters: 0.4098248346207552
Coherence score with 13 clusters: 0.40824833050236037
Coherence score with 14 clusters: 0.3787646649261011
Coherence score with 15 clusters: 0.3960207923587834
Coherence score with 16 clusters: 0.36823008276700264
Coherence score with 17 clusters: 0.365346421609319
Coherence score with 18 clusters: 0.3518221635663537
Coherence score with 19 clusters: 0.34411307510487993
Coherence score with 20 clusters: 0.37443521155656795


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=6, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.341*"pessoa" + 0.260*"falar" + 0.179*"virus" + 0.175*"bolsonaro" + 0.152*"gente" + 0.151*"querer" + 0.144*"saber" + 0.133*"agua" + 0.131*"tomar" + 0.120*"ficar".
Words in 1: 0.437*"bolsonaro" + -0.286*"agua" + -0.247*"falar" + -0.175*"cafe" + -0.164*"tomar" + 0.159*"moro" + -0.155*"virus" + -0.151*"pessoa" + 0.143*"brasileiro" + -0.141*"sangue".
Words in 2: -0.434*"falar" + -0.348*"cafe" + -0.343*"bolsonaro" + 0.215*"pessoa" + -0.180*"sangue" + -0.167*"tomar" + 0.161*"virus" + -0.102*"moro" + -0.095*"descarregar" + -0.095*"saber".
Words in 3: 0.473*"agua" + -0.270*"pegar" + -0.206*"entender" + -0.164*"mascara" + -0.160*"morrer" + -0.157*"querer" + 0.145*"corpo" + -0.123*"cara" + -0.117*"pega_mascara" + 0.109*"banho".
Words in 4: -0.280*"agua" + 0.245*"cafe" + -0.215*"bolsonaro" + -0.214*"pegar" + -0.170*"colocar" + 0.153*"falar" + 0.151*"cadastro" + 0.150*"site" + 0.144*"novo" + 0.135*"tomar".
Words in 5: -0.366*"pessoa" + 0.246*"virus" + 0.191*"mundial" + 0.175*"chinês" 

## Método LSA - APLICAÇÃO NO DATASET DE 2021

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = df_2021.message_norm_treatment_ssw2.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['mario', 'sergio', 'cortella', 'vais', 'andando', 'xicara', 'cafe', 'repente', 'empurra', 'derrames', 'cafe', 'lado', 'derramaste', 'cafe', 'empurrou', 'resposta', 'errada', 'derramaste', 'cafe', 'tinhas', 'cafe', 'caneca', 'terias', 'derramado', 'tiveres', 'xicara', 'derramar', 'portanto', 'vida', 'sacode']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['andar', 'xicara', 'cafe', 'repente', 'empurrar', 'derrame', 'cafe', 'lado', 'derramaste', 'cafe', 'empurrar', 'resposta', 'errar', 'cafe', 'tinhas', 'cafe', 'caneca', 'terias', 'derramar', 'tiver', 'xicara', 'derramar', 'vida', 'sacoder', 'tiver', 'dentro', 'derramar', 'podes', 'vida', 'fingir']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.37957942760243646
Coherence score with 5 clusters: 0.4146550953346024
Coherence score with 6 clusters: 0.47597472723500217
Coherence score with 7 clusters: 0.4394051498631172
Coherence score with 8 clusters: 0.36140864173757714
Coherence score with 9 clusters: 0.43220260754085704
Coherence score with 10 clusters: 0.38667968038013345
Coherence score with 11 clusters: 0.36827747131551103
Coherence score with 12 clusters: 0.3753148811869203
Coherence score with 13 clusters: 0.3622325687810856
Coherence score with 14 clusters: 0.3914975936486547
Coherence score with 15 clusters: 0.35450709548671744
Coherence score with 16 clusters: 0.40052355950368923
Coherence score with 17 clusters: 0.36149399037868934
Coherence score with 18 clusters: 0.3847828392371386
Coherence score with 19 clusters: 0.4007233465742729
Coherence score with 20 clusters: 0.41245460553770397


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=6, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: -0.235*"pessoa" + -0.191*"querer" + -0.183*"gente" + -0.177*"presidente" + -0.163*"bolsonaro" + -0.156*"tomar" + -0.148*"vacino" + -0.141*"ficar" + -0.141*"governo" + -0.130*"covid".
Words in 1: 0.351*"presidente" + 0.304*"bolsonaro" + 0.203*"governo" + -0.182*"gente" + -0.179*"covid" + -0.171*"agua" + 0.164*"brasileiro" + -0.155*"pessoa" + 0.142*"governador" + 0.131*"vacino".
Words in 2: 0.399*"vacino" + 0.258*"tomar" + 0.224*"dose" + -0.190*"presidente" + 0.182*"vacina" + 0.169*"pessoa" + 0.156*"producao" + -0.142*"bolsonaro" + -0.130*"covid" + -0.126*"querer".
Words in 3: 0.267*"governo" + -0.238*"presidente" + 0.208*"oxigenio" + 0.149*"governador" + 0.147*"producao" + -0.137*"tomar" + -0.136*"politico" + -0.126*"acabar" + 0.123*"dose" + 0.121*"hospital".
Words in 4: 0.343*"bolsonaro" + -0.285*"oxigenio" + -0.210*"governador" + -0.187*"pessoa" + 0.166*"covid" + 0.142*"medico" + 0.140*"receber" + 0.139*"entregar" + 0.120*"governo_federal" + -0.120*"agua".
Words in 5: -0.2

## Método LSA - APLICAÇÃO NO DATASET DE POLÍTICA

In [ ]:
tm_politica

2       ['novo', 'mega', 'campeao', 'brasil', 'enrique...
3       ['lulinha', 'filho', 'lula', 'comprou', 'aviao...
5       ['manda', 'nesse', 'pais', 'sento', 'quiser', ...
6       ['mensagem', 'falsa', 'conta', 'briga', 'luiz'...
20      ['foto', 'dilma', 'rousseff', 'fuzil', 'montag...
                              ...                        
5176    ['estadao', 'exclusivo', 'farsa', 'revelada', ...
5179    ['urgente', 'exercito', 'determina', 'voto', '...
5189    ['vaza', 'video', 'joice', 'hasselmann', 'mach...
5190    ['bolsonaro', 'casamento', 'isolada', 'ex-mulh...
5195    ['gente', 'velho', 'luladrao', 'endoideceu', '...
Name: message_norm_treatment_ssw2, Length: 1508, dtype: object

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_politica.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['novo', 'mega', 'campeao', 'brasil', 'enriquecimento', 'subito', 'proprietario', 'fazenda', 'fazenda', 'fortaleza', 'comprada', 'certificada', 'cartorio', 'registro', 'imoveis', 'proprietario', 'fabio', 'luis', 'lula', 'silva', 'propriedade', 'fazenda', 'regiao', 'valparaiso', 'sppreco', 'milhoes', 'reaispois', 'lulinha', 'filho', 'presidente']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['novo', 'mega', 'subito', 'proprietario', 'fazenda', 'fazenda', 'fortaleza', 'comprar', 'certificar', 'cartorio', 'registro', 'imovel', 'proprietario', 'propriedade', 'fazenr', 'valparaiso', 'reaispois', 'Filho', 'presidente', 'ano', 'subempregar', 'zoologico', 'acabar', 'comprar', 'fazenda', 'fortalezar', 'porteira', 'fechar', 'localizar', 'margem']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.42332385382597004
Coherence score with 5 clusters: 0.36535562328649485
Coherence score with 6 clusters: 0.379086074229179
Coherence score with 7 clusters: 0.38505650019069043
Coherence score with 8 clusters: 0.43354323899422315
Coherence score with 9 clusters: 0.37628461714772515
Coherence score with 10 clusters: 0.41297152288151856
Coherence score with 11 clusters: 0.36331394871685146
Coherence score with 12 clusters: 0.38358330929300827
Coherence score with 13 clusters: 0.42139258505608895
Coherence score with 14 clusters: 0.40377741253127863
Coherence score with 15 clusters: 0.36104569173578477
Coherence score with 16 clusters: 0.37800386974505584
Coherence score with 17 clusters: 0.366903503801818
Coherence score with 18 clusters: 0.36818699789399045
Coherence score with 19 clusters: 0.37710462767028824
Coherence score with 20 clusters: 0.35615770026570864


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=8, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.408*"bolsonaro" + 0.310*"presidente" + 0.175*"governo" + 0.158*"acabar" + 0.147*"querer" + 0.143*"pessoa" + 0.140*"brasileiro" + 0.129*"saber" + 0.124*"falar" + 0.120*"pai".
Words in 1: 0.726*"bolsonaro" + -0.248*"acabar" + -0.139*"ano" + -0.133*"pessoa" + -0.108*"pai" + -0.101*"dinheiro" + -0.098*"aumento" + -0.097*"governo" + -0.095*"salario" + -0.095*"falar".
Words in 2: -0.562*"presidente" + 0.434*"acabar" + 0.268*"bolsonaro" + 0.171*"aumento" + 0.153*"partido" + -0.151*"governo" + 0.130*"aprovar" + 0.119*"pessoa" + -0.113*"brasileiro" + -0.102*"temer".
Words in 3: -0.345*"globo" + 0.294*"presidente" + -0.246*"temer" + -0.239*"falar" + -0.238*"jornalismo" + -0.203*"noticia" + -0.201*"alinhar" + -0.200*"rede_globo" + -0.133*"jornal" + 0.126*"governo".
Words in 4: 0.517*"presidente" + -0.450*"governo" + 0.268*"acabar" + -0.185*"federal" + 0.124*"partido" + -0.121*"mandar" + 0.118*"aumento" + -0.111*"brasileiro" + -0.110*"governador" + -0.102*"medico".
Words in 5: -0.426

## Método LSA - APLICAÇÃO NO DATASET DE BRASIL

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_brasil.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['natanael', 'bufalo', 'matou', 'menina', 'anos', 'foragido']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['natanael', 'bufalo', 'matar', 'menina', 'ano', 'foragido']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.48646096263285066
Coherence score with 5 clusters: 0.49797109354731584
Coherence score with 6 clusters: 0.43564905195078946
Coherence score with 7 clusters: 0.48339193246033674
Coherence score with 8 clusters: 0.45578814257481587
Coherence score with 9 clusters: 0.4372824166144031
Coherence score with 10 clusters: 0.4463888842855395
Coherence score with 11 clusters: 0.48274914620831294
Coherence score with 12 clusters: 0.4317094517291014
Coherence score with 13 clusters: 0.40159607544650994
Coherence score with 14 clusters: 0.39184292025032647
Coherence score with 15 clusters: 0.41099393982388543
Coherence score with 16 clusters: 0.3955484093099752
Coherence score with 17 clusters: 0.38906705705905875
Coherence score with 18 clusters: 0.3918446811557395
Coherence score with 19 clusters: 0.3749767019883104
Coherence score with 20 clusters: 0.3807422776205391


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=5, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: -0.235*"falar" + -0.217*"pessoa" + -0.186*"passar" + -0.184*"ficar" + -0.159*"saber" + -0.140*"ano" + -0.137*"gente" + -0.132*"casa" + -0.127*"acontecer" + -0.126*"crianca".
Words in 1: -0.411*"falar" + -0.353*"filha" + -0.277*"senhor" + -0.193*"shopping" + -0.155*"juntar" + -0.155*"estrutura" + -0.152*"forte" + 0.151*"passar" + -0.145*"mensagem" + 0.126*"crianca".
Words in 2: -0.292*"forca" + -0.240*"area" + -0.171*"auxiliar" + -0.154*"delegar" + -0.148*"aereo" + -0.148*"partir" + 0.137*"crianca" + -0.135*"estadual" + -0.131*"intervencao_militar" + -0.128*"secretario".
Words in 3: -0.409*"desconto" + -0.384*"passagem" + -0.373*"aereo" + -0.317*"idoso" + -0.276*"empresa" + 0.127*"forca" + -0.106*"ano" + -0.101*"grande" + -0.099*"adquirir" + 0.094*"area".
Words in 4: 0.344*"crianca" + -0.227*"governo" + -0.219*"receber" + 0.218*"matar" + -0.162*"saber" + -0.153*"brasileiro" + -0.149*"trabalhar" + 0.147*"ficar" + 0.138*"aereo" + 0.136*"desconto".


## Método LSA - APLICAÇÃO NO DATASET DE SAÚDE

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_saude.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['alerta', 'geral', 'atencao', 'fanta', 'propaganda', 'parouporque', 'reparem', 'propaganda', 'quase', 'midia', 'repassando', 'mail', 'abaixo', 'conhecimento', 'prevencao', 'principalmente', 'bebem', 'refrigerante', 'fanta', 'uva', 'este', 'mail', 'repassado', 'dentro', 'hospital', 'trabalha', 'pessoa', 'amiga', 'fato', 'confirmado']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['alertar', 'geral', 'atencao', 'fanto', 'propaganda', 'parouporque', 'repar', 'propaganda', 'quase', 'midia', 'repassar', 'mail', 'abaixo', 'conhecimento', 'prevencao', 'principalmente', 'beber', 'refrigerante', 'fanta', 'uva', 'mail', 'repassar', 'dentro', 'hospital', 'trabalhar', 'pessoa', 'amigo', 'fato', 'confirmar', 'tres']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.4192804304649701
Coherence score with 5 clusters: 0.4476327092881651
Coherence score with 6 clusters: 0.4131639973716286
Coherence score with 7 clusters: 0.42545876871358207
Coherence score with 8 clusters: 0.4066632280760556
Coherence score with 9 clusters: 0.36246819484884524
Coherence score with 10 clusters: 0.3834141873959457
Coherence score with 11 clusters: 0.37996834197987
Coherence score with 12 clusters: 0.3959865199863026
Coherence score with 13 clusters: 0.3646531229139436
Coherence score with 14 clusters: 0.38714073532665144
Coherence score with 15 clusters: 0.4000353232697763
Coherence score with 16 clusters: 0.3807732143295258
Coherence score with 17 clusters: 0.3843909549916853
Coherence score with 18 clusters: 0.379675227364975
Coherence score with 19 clusters: 0.3853108977117394
Coherence score with 20 clusters: 0.3649963279499212


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=5, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.347*"pessoa" + 0.254*"falar" + 0.241*"virus" + 0.238*"tomar" + 0.210*"gente" + 0.195*"agua" + 0.130*"pegar" + 0.129*"vacino" + 0.125*"causar" + 0.117*"passar".
Words in 1: 0.260*"causar" + -0.250*"falar" + 0.241*"zika" + -0.221*"pegar" + 0.185*"virus" + -0.181*"gente" + 0.175*"vacina" + -0.174*"tomar" + 0.164*"quimico" + -0.162*"mascara".
Words in 2: 0.550*"agua" + -0.278*"pegar" + -0.247*"mascara" + 0.161*"falar" + 0.160*"tomar" + -0.119*"entender" + -0.116*"governo" + 0.107*"sangue" + 0.099*"banho" + 0.099*"corpo".
Words in 3: 0.365*"agua" + -0.361*"tomar" + 0.323*"pegar" + 0.267*"mascara" + -0.266*"vacino" + 0.192*"virus" + 0.144*"entender" + -0.129*"cafe" + -0.117*"caso" + -0.114*"ficar".
Words in 4: -0.291*"falar" + -0.264*"cafe" + -0.226*"tomar" + 0.222*"comer" + 0.178*"fruta" + -0.173*"virus" + 0.151*"pessoa" + -0.145*"vacino" + 0.135*"hospital" + -0.134*"sangue".


## Método LSA - APLICAÇÃO NO DATASET DE ENTRETENIMENTO

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_entretenimento.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['simpsons', 'episodio', 'manifestacoes', 'brasil']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['simpsons', 'episodio', 'Manifestacoes']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.44730350330226704
Coherence score with 5 clusters: 0.4065365337191934
Coherence score with 6 clusters: 0.4056437966988479
Coherence score with 7 clusters: 0.38839957162518196
Coherence score with 8 clusters: 0.41994172473346913
Coherence score with 9 clusters: 0.4122892557449871
Coherence score with 10 clusters: 0.4205928187837372
Coherence score with 11 clusters: 0.45206853500926447
Coherence score with 12 clusters: 0.42862940968349
Coherence score with 13 clusters: 0.4472102128438412
Coherence score with 14 clusters: 0.3999109795559025
Coherence score with 15 clusters: 0.42222136204088134
Coherence score with 16 clusters: 0.41770937909065853
Coherence score with 17 clusters: 0.41858797569427314
Coherence score with 18 clusters: 0.42342848963523666
Coherence score with 19 clusters: 0.430847797771955
Coherence score with 20 clusters: 0.4340849681832187


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=11, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: -0.212*"querer" + -0.188*"pessoa" + -0.177*"morrer" + -0.175*"globo" + -0.171*"ano" + -0.169*"ficar" + -0.165*"vida" + -0.160*"hoje" + -0.142*"programa" + -0.141*"deixar".
Words in 1: 0.411*"globo" + 0.270*"programa" + -0.192*"vida" + -0.186*"hoje" + 0.170*"bolsonaro" + -0.166*"aprender" + -0.161*"dizer" + -0.142*"tempo" + 0.133*"rede" + -0.130*"pensar".
Words in 2: 0.303*"gente" + 0.266*"cara" + 0.240*"mundo" + 0.208*"pessoa" + -0.172*"luciano_huck" + -0.147*"investimento" + -0.146*"chegar" + -0.144*"globo" + 0.143*"querer" + 0.127*"discutir".
Words in 3: 0.310*"luciano_huck" + 0.269*"investimento" + 0.226*"produto" + 0.220*"formular" + -0.206*"globo" + 0.176*"amostra" + 0.149*"pessoa" + 0.132*"tornar" + 0.129*"pesquisa" + -0.128*"ano".
Words in 4: 0.343*"ano" + 0.325*"morrer" + -0.220*"globo" + 0.190*"ator" + 0.170*"cantor" + -0.153*"dizer" + -0.142*"bolsonaro" + -0.141*"programa" + -0.136*"deixar" + 0.133*"falar".
Words in 5: 0.319*"programa" + -0.318*"presidente" + 0.29

## Método LSA - APLICAÇÃO NO DATASET DE TECNOLOGIA

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_tecnologia.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['virus', 'homem', 'teste', 'fidelidade', 'descobre', 'mulher', 'lesbica']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['virus', 'homem', 'teste', 'fidelidade', 'descobrir', 'mulher', 'lesbico']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.5643663406857131
Coherence score with 5 clusters: 0.47919837328647086
Coherence score with 6 clusters: 0.6435346468230966
Coherence score with 7 clusters: 0.5609296287818822
Coherence score with 8 clusters: 0.6338460075907174
Coherence score with 9 clusters: 0.5771555313169994
Coherence score with 10 clusters: 0.5318536580266799
Coherence score with 11 clusters: 0.47316419548053407
Coherence score with 12 clusters: 0.44963546253225584
Coherence score with 13 clusters: 0.5356931596664187
Coherence score with 14 clusters: 0.49530808754377553
Coherence score with 15 clusters: 0.46686346374819876
Coherence score with 16 clusters: 0.4428594963445109
Coherence score with 17 clusters: 0.4387840482565547
Coherence score with 18 clusters: 0.4665403613500561
Coherence score with 19 clusters: 0.4309641828689344
Coherence score with 20 clusters: 0.4038824897865164


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=6, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.310*"mensagem" + 0.285*"site" + 0.255*"receber" + 0.215*"compartilhar" + 0.213*"contato" + 0.212*"amigo" + 0.199*"abaixo" + 0.184*"ganhar" + 0.162*"presente" + 0.158*"novo".
Words in 1: -0.479*"mensagem" + 0.291*"site" + -0.252*"contato" + -0.214*"enviar" + 0.197*"abaixo" + 0.166*"compartilhar" + 0.160*"ganhar" + 0.155*"presente" + 0.151*"cadastro" + -0.118*"celular".
Words in 2: -0.372*"cadastro" + 0.346*"presente" + -0.268*"informar" + -0.264*"site" + -0.251*"beneficio" + 0.188*"premio" + 0.177*"abaixo" + 0.167*"compartilhar" + -0.153*"programa" + 0.145*"novo".
Words in 3: -0.419*"atualizar" + 0.243*"presente" + -0.203*"bloquear" + 0.189*"contato" + 0.189*"mensagem" + -0.157*"repassar" + -0.148*"amigo" + -0.145*"deixar" + -0.137*"pedir" + -0.135*"ficar".
Words in 4: 0.405*"presente" + 0.311*"atualizar" + -0.221*"compartilhar" + 0.196*"novo" + -0.193*"site" + -0.187*"mensagem" + -0.162*"abaixo" + 0.161*"cadastro" + 0.153*"premio" + -0.139*"retirar".
Words in 5: 0.325*"at

## Método LSA - APLICAÇÃO NO DATASET DE CIÊNCIA

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_ciencia.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['teoria', 'falsa', 'masaru', 'emoto', 'palavras', 'alterar', 'substancias']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['teoria', 'falso', 'Masaru', 'emoto', 'palavra', 'alterar', 'substancia']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.7016859169633893
Coherence score with 5 clusters: 0.6611656309758628
Coherence score with 6 clusters: 0.786900325945144
Coherence score with 7 clusters: 0.7411912482677465
Coherence score with 8 clusters: 0.7345264111578247
Coherence score with 9 clusters: 0.6771686291311144
Coherence score with 10 clusters: 0.6156056335724427
Coherence score with 11 clusters: 0.6090114855753326
Coherence score with 12 clusters: 0.6290677457029797
Coherence score with 13 clusters: 0.6644855918007239
Coherence score with 14 clusters: 0.6824607119297613
Coherence score with 15 clusters: 0.6548464670434727
Coherence score with 16 clusters: 0.6026125182596254
Coherence score with 17 clusters: 0.6122118318133627
Coherence score with 18 clusters: 0.5906994419218977
Coherence score with 19 clusters: 0.615101994829478
Coherence score with 20 clusters: 0.6536620908126173


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=6, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.343*"espelho" + 0.206*"dia" + 0.194*"insolacao" + 0.160*"ficar" + 0.156*"casa" + 0.152*"poder" + 0.145*"dentro" + 0.142*"fenomeno" + 0.136*"temperatura" + 0.131*"perigoso".
Words in 1: 0.534*"espelho" + 0.166*"parte" + 0.165*"espaco" + 0.164*"refletir" + 0.164*"unha" + -0.154*"dia" + -0.146*"insolacao" + 0.129*"mulher" + 0.126*"imagem" + 0.123*"teste".
Words in 2: 0.478*"terra" + 0.297*"climatico" + 0.231*"natural" + 0.161*"causar" + 0.149*"orbitar" + 0.149*"mudanca" + 0.126*"hoje" + 0.126*"nasa" + 0.120*"depender" + 0.119*"ano".
Words in 3: -0.422*"natural" + 0.257*"climatico" + 0.237*"terra" + -0.168*"corona" + -0.167*"espalhar" + -0.166*"professor" + -0.163*"dizer" + 0.129*"orbitar" + 0.129*"mudanca" + -0.127*"afetar".
Words in 4: 0.316*"pulso" + 0.316*"sangue" + 0.241*"cientista" + 0.237*"frasco" + 0.168*"conseguir" + 0.166*"morte" + 0.163*"soro" + 0.158*"fracassar" + 0.158*"valvula" + 0.158*"vasilha".
Words in 5: -0.203*"catastrofe" + -0.173*"ilha" + 0.140*"corona" +

## Método LSA - APLICAÇÃO NO DATASET DE ESPORTE

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_esporte.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['virus', 'logotipo', 'copa', 'mundo', 'mensagem', 'subliminar']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['virus', 'logotipo', 'Copa', 'mundo', 'mensagem', 'subliminar']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.6926411294178936
Coherence score with 5 clusters: 0.5439153705821173
Coherence score with 6 clusters: 0.5379491345949124
Coherence score with 7 clusters: 0.5044488841617132
Coherence score with 8 clusters: 0.5105002127864703
Coherence score with 9 clusters: 0.612562662639462
Coherence score with 10 clusters: 0.508479674690671
Coherence score with 11 clusters: 0.5978102621842779
Coherence score with 12 clusters: 0.5572864681383634
Coherence score with 13 clusters: 0.5133071106719774
Coherence score with 14 clusters: 0.5100816159411895
Coherence score with 15 clusters: 0.5958632511992246
Coherence score with 16 clusters: 0.5106851336775186
Coherence score with 17 clusters: 0.4925257037450193
Coherence score with 18 clusters: 0.5454918387552564
Coherence score with 19 clusters: 0.4453075259766949
Coherence score with 20 clusters: 0.49287418540867955


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=4, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: -0.346*"jogador" + -0.230*"atleta" + -0.205*"jogo" + -0.201*"flamengo" + -0.197*"presidente" + -0.189*"querer" + -0.178*"falar" + -0.160*"brasileiro" + -0.154*"ficar" + -0.151*"americano".
Words in 1: -0.498*"jogador" + 0.229*"falar" + 0.187*"atleta" + 0.169*"querer" + 0.161*"americano" + -0.142*"flamengo" + -0.137*"mundo" + -0.137*"tecnico" + 0.124*"ficar" + 0.123*"achar".
Words in 2: 0.321*"atleta" + -0.291*"falar" + -0.220*"jogo" + 0.199*"americano" + -0.169*"terrorista" + -0.165*"gente" + 0.156*"hotel" + -0.150*"casa" + -0.131*"morar" + -0.127*"mineirao".
Words in 3: 0.257*"flamengo" + -0.203*"presidente" + -0.198*"gesto" + 0.179*"americano" + -0.148*"brasileiro" + 0.143*"hotel" + 0.139*"jovem" + -0.136*"falar" + 0.125*"jogar" + 0.121*"saber".


## Método LSA - APLICAÇÃO NO DATASET DE MUNDO

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_mundo.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['mentira', 'chineses', 'comem', 'bebes', 'aumentar', 'desempenho']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['mentira', 'chinês', 'comer', 'bebe', 'aumentar', 'desempenho']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.5302977201330468
Coherence score with 5 clusters: 0.44448496718102176
Coherence score with 6 clusters: 0.4446914225836162
Coherence score with 7 clusters: 0.4242830938919813
Coherence score with 8 clusters: 0.3946384456220431
Coherence score with 9 clusters: 0.4067097367546177
Coherence score with 10 clusters: 0.39065709294480977
Coherence score with 11 clusters: 0.47517214395953306
Coherence score with 12 clusters: 0.4473155785080758
Coherence score with 13 clusters: 0.4616761143658505
Coherence score with 14 clusters: 0.4139656808732341
Coherence score with 15 clusters: 0.4612763066068053
Coherence score with 16 clusters: 0.48788183251148165
Coherence score with 17 clusters: 0.5280219867952474
Coherence score with 18 clusters: 0.46075469891939075
Coherence score with 19 clusters: 0.4469459933103468
Coherence score with 20 clusters: 0.5084582806013785


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=4, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.289*"pessoa" + 0.246*"mundo" + 0.181*"pai" + 0.164*"ano" + 0.135*"homem" + 0.127*"governo" + 0.123*"virus" + 0.116*"mulher" + 0.113*"apenas" + 0.107*"direito".
Words in 1: -0.345*"mulher" + 0.247*"virus" + -0.213*"direito" + -0.199*"pessoa" + -0.155*"considerar" + 0.155*"chinês" + 0.134*"morcego" + 0.129*"riqueza" + 0.127*"rato" + 0.124*"mundial".
Words in 2: -0.471*"mulher" + -0.218*"direito" + 0.169*"pessoa" + -0.168*"considerar" + -0.144*"especie" + 0.142*"pai" + -0.130*"veredicto" + -0.127*"virus" + 0.122*"muculmano" + -0.111*"especialista".
Words in 3: 0.526*"pessoa" + -0.215*"muculmano" + -0.171*"pai" + -0.162*"canada" + -0.154*"mulher" + -0.152*"prefeito" + 0.143*"golfinho" + -0.133*"entender" + -0.118*"vida" + -0.109*"falso".


## Método LSA - APLICAÇÃO NO DATASET DE RELIGIÃO

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_religiao.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['pastor', 'sergio', 'helder', 'chutou', 'santa', 'virou', 'catolico']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['pastor', 'chutar', 'virar', 'catolico']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.5476353881334544
Coherence score with 5 clusters: 0.6038388392219703
Coherence score with 6 clusters: 0.521625755765372
Coherence score with 7 clusters: 0.5074387433451472
Coherence score with 8 clusters: 0.4958741509612396
Coherence score with 9 clusters: 0.4368844024347227
Coherence score with 10 clusters: 0.49974577985976526
Coherence score with 11 clusters: 0.4116575206344999
Coherence score with 12 clusters: 0.4438987174527791
Coherence score with 13 clusters: 0.4174249799860984
Coherence score with 14 clusters: 0.4106505154465569
Coherence score with 15 clusters: 0.4447962318518909
Coherence score with 16 clusters: 0.4047565894442384
Coherence score with 17 clusters: 0.40465878903180624
Coherence score with 18 clusters: 0.40826220286784987
Coherence score with 19 clusters: 0.36292574088485297
Coherence score with 20 clusters: 0.3881659468451935


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=5, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.256*"pessoa" + 0.253*"falar" + 0.172*"querer" + 0.170*"deus" + 0.168*"igreja" + 0.157*"saber" + 0.156*"mensagem" + 0.146*"senhor" + 0.143*"orar" + 0.131*"pedir".
Words in 1: -0.398*"falar" + -0.300*"bolsonaro" + -0.286*"mudanca" + -0.180*"querer" + -0.174*"saber" + -0.154*"usar" + 0.153*"mensagem" + -0.152*"deus" + -0.144*"enviar" + 0.124*"ajudar".
Words in 2: 0.350*"cama" + 0.271*"gazin" + 0.231*"terra" + 0.210*"comprar" + -0.210*"mensagem" + 0.193*"saquinho" + 0.155*"cemiterio" + 0.136*"tirar" + 0.131*"video" + -0.122*"povo".
Words in 3: 0.220*"ajudar" + -0.187*"irmao" + -0.180*"povo" + 0.173*"colocar" + 0.171*"gente" + -0.167*"poder" + 0.163*"vida" + -0.160*"mensagem" + -0.156*"senhor" + 0.151*"crianca".
Words in 4: -0.244*"senhor" + -0.235*"igreja" + 0.200*"encher" + 0.198*"cama" + 0.181*"jejum" + 0.169*"mensagem" + -0.160*"foto" + 0.153*"gazin" + 0.140*"pessoa" + -0.138*"orar".


## Método LSA - APLICAÇÃO NO DATASET GERAL

In [ ]:
stop_words = stopwords.words('portuguese')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = tm_geral.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['simpsons', 'episodio', 'manifestacoes', 'brasil']


In [ ]:
# Cria os modelos de bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # Quanto maior o limite, menos frases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Removendo as stopwords
data_words_nostops = remove_stopwords(data_words)

# Formando os bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# Inicializando o modelo spacy 'pt', mantendo apenas o componente tagger (para eficiência)
nlp = spacy.load("pt_core_news_lg", disable=['parser', 'ner'])

# Fazendo lematização mantendo apenas substantivo, adjetivo, verbo e advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['simpsons', 'episodio', 'Manifestacoes']


In [ ]:
# Criando dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Criando Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [ ]:
# Achando o valor de coerência com diferentes números de tópicos
for i in range(4,21):
    lsi = LsiModel(bow, num_topics=i, id2word=id2word)
    coherence_model = CoherenceModel(model=lsi, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 4 clusters: 0.45916256326687405
Coherence score with 5 clusters: 0.4667496906793943
Coherence score with 6 clusters: 0.44434570128737577
Coherence score with 7 clusters: 0.4555948407139419
Coherence score with 8 clusters: 0.4992460775590711
Coherence score with 9 clusters: 0.4293939572740871
Coherence score with 10 clusters: 0.4654567232964867
Coherence score with 11 clusters: 0.4147429902820432
Coherence score with 12 clusters: 0.43012300830513434
Coherence score with 13 clusters: 0.43299585917401023
Coherence score with 14 clusters: 0.4126778388797588
Coherence score with 15 clusters: 0.43229668438944646
Coherence score with 16 clusters: 0.4106512479286109
Coherence score with 17 clusters: 0.406729690972846
Coherence score with 18 clusters: 0.4290681593248755
Coherence score with 19 clusters: 0.4057014273617947
Coherence score with 20 clusters: 0.40327662578207935


In [ ]:
# Perfomando SVD na bag of words com o LsiModel para extrair o número ótimo de tópicos
lsi = LsiModel(bow, num_topics=8, id2word=id2word)

In [ ]:
# Achando as 10 palavras com maior associação com os tópicos derivados
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.294*"pessoa" + 0.228*"falar" + 0.170*"querer" + 0.158*"saber" + 0.155*"gente" + 0.155*"ficar" + 0.138*"passar" + 0.131*"bolsonaro" + 0.130*"presidente" + 0.129*"ano".
Words in 1: 0.491*"bolsonaro" + -0.358*"pessoa" + 0.355*"presidente" + -0.231*"falar" + 0.210*"governo" + 0.202*"brasileiro" + -0.145*"tomar" + -0.135*"gente" + -0.126*"agua" + -0.113*"virus".
Words in 2: -0.553*"falar" + -0.298*"bolsonaro" + 0.201*"pessoa" + -0.189*"gente" + -0.175*"tomar" + 0.169*"receber" + 0.161*"mensagem" + -0.154*"querer" + 0.121*"ano" + -0.115*"presidente".
Words in 3: 0.545*"pessoa" + 0.447*"bolsonaro" + -0.246*"ano" + 0.127*"virus" + -0.126*"querer" + 0.125*"presidente" + -0.124*"ficar" + -0.124*"mulher" + -0.122*"filho" + 0.122*"mensagem".
Words in 4: 0.369*"mensagem" + 0.283*"bolsonaro" + -0.228*"governo" + -0.219*"presidente" + -0.181*"ano" + 0.170*"amigo" + 0.154*"contato" + -0.151*"brasileiro" + -0.149*"vacino" + 0.138*"compartilhar".
Words in 5: 0.414*"falar" + -0.259*"bolsona